In [2]:
# imports

import numpy as np
import rasterio
import laspy
import pandas as pd
import matplotlib.pyplot as plt
import lazrs
from pathlib import Path
from laspy import LazBackend

# Pick 'lazrs' if you installed it, or 'laszip' if that's what you've got
# laspy.set_laz_backend(LazBackend.Lazrs)
# laspy.file.

In [3]:
###############################################################################
# 1. QUICK METRICS FROM THE CHM (GeoTIFF, 1 m pixels)
###############################################################################

def chm_summary(chm_path: Path):
    """Return % woody cover > 1 m, mean height, P90 height."""
    with rasterio.open(chm_path) as src:
        chm = src.read(1, masked=True)          # 2‑D masked array, metres
    woody_mask = chm > 1                        # dense‑woody threshold
    pct_woody = woody_mask.mean() * 100         # % of raster area
    mean_h     = float(chm.mean())              # metres
    p90_h      = float(np.percentile(chm.compressed(), 90))
    return pct_woody, mean_h, p90_h

In [4]:
###############################################################################
# 2. STEM‑DENSITY PROXY FROM THE POINT CLOUD (HeightAboveGround already added)
###############################################################################

def stem_density(las_path: Path, grid=2.0):
    """
    Very lightweight stem‑density proxy:
      • keep points with HeightAboveGround > 2 m (ignore grass/shrub noise)
      • drop them into a grid (default 2 × 2 m)
      • assume one stem/cluster per occupied cell
      • report stems ha⁻¹
    """
    las = laspy.read(las_path, laz_backend=LazBackend.Lazrs)
    try:
        hag = las['HeightAboveGround']          # PDAL wrote this extra dim
    except KeyError:
        raise RuntimeError("LAS file missing HeightAboveGround dimension")

    mask = hag > 2                              # metres
    if mask.sum() == 0:
        return 0.0

    x = las.x[mask]
    y = las.y[mask]

    # snap to grid
    gx = np.floor((x - x.min()) / grid).astype(np.int32)
    gy = np.floor((y - y.min()) / grid).astype(np.int32)
    occupied = np.unique(np.stack([gx, gy], axis=1), axis=0).shape[0]

    # area covered = raster footprint (min→max) so density is consistent
    area_m2 = (x.max() - x.min()) * (y.max() - y.min())
    area_ha = area_m2 / 10_000
    return occupied / area_ha                  # stems per hectare

In [5]:
###############################################################################
# 3. WRAP EVERYTHING FOR ONE DATE (EXTEND TO MANY DATES AS NEEDED)
###############################################################################

def analyse_block(date, chm_path, las_path):
    pct, mean_h, p90_h = chm_summary(chm_path)
    stems              = stem_density(las_path)
    return {
        "date": pd.to_datetime(date),
        "woody_cover_pct": pct,
        "mean_height_m": mean_h,
        "p90_height_m": p90_h,
        "stem_density_stems_per_ha": stems,
    }

In [6]:
block = analyse_block(
    date="2025-03-01",
    chm_path=Path("output\\chm_0_25m.tif"),
    las_path=Path("output\\rehab_sample_hag.laz"),
)

block


{'date': Timestamp('2025-03-01 00:00:00'),
 'woody_cover_pct': 9.54538570622169,
 'mean_height_m': 0.2710962679691449,
 'p90_height_m': 0.9739999771118164,
 'stem_density_stems_per_ha': 179.43209466842882}

In [6]:
las = laspy.read("output\\rehab_sample_hag.las")
list(las.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'synthetic',
 'key_point',
 'withheld',
 'overlap',
 'scanner_channel',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'user_data',
 'scan_angle',
 'point_source_id',
 'gps_time',
 'HeightAboveGround']

In [7]:
las = laspy.read("input\\unfiltered\\291000_6424000.las")
list(las.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'synthetic',
 'key_point',
 'withheld',
 'overlap',
 'scanner_channel',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'user_data',
 'scan_angle',
 'point_source_id',
 'gps_time',
 'Amplitude',
 'Reflectance',
 'Deviation']